In [1]:
cd ..

/home/pi/WORK_DIR/notebooks/Ex_Frank/LABS


In [81]:
pwd

'/home/pi/WORK_DIR/notebooks/Ex_Frank/LABS'

# 1.1

In [107]:
from Lab3.lab3_ex1 import WindowGenerator
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import time

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, help='model name among mlp cnn and lstm')
parser.add_argument('--labels', type=int, default=0,
                    help='0 for temp forecasting, 1 for hum forecasting, 2 or more for both')
#parser.add_argument('--saved_model_dir', type=str, default=None)
args = parser.parse_args(['--labels=2'])

In [11]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = "Lab3/data/jena_climate_2009_2016.csv.zip"

csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

n = len(data)
train_data = data[0:int(n * 0.7)]
val_data = data[int(n * 0.7):int(n * 0.9)]
test_data = data[int(n * 0.9):]

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

input_width = 6
LABEL_OPTIONS = args.labels

generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

# Let's save datasets on disk
tf.data.experimental.save(train_ds, './Lab4/th_train')
tf.data.experimental.save(val_ds, './Lab4/th_val')
tf.data.experimental.save(test_ds, './Lab4/th_test')

To load a previously saved dataset, you need to specify element_spec, a type signature of the elements of the saved dataset, which can be obtained via tf.data.Dataset.element_spec. This requirement exists so that shape inference of the loaded dataset does not need to perform I/O.



In [12]:
tensor_specs = (tf.TensorSpec([None, 6, 2], dtype=tf.float32),
                tf.TensorSpec([None, 2]))
#train_ds = tf.data.experimental.load('./Lab4/th_train', tensor_specs)
#val_ds = tf.data.experimental.load('./Lab4/th_val', tensor_specs)
#test_ds = tf.data.experimental.load('./Lab4/th_test', tensor_specs)

# 1.2

In [139]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/model_{}.tflite".format("cnn_3"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_conv1d_1_input:0
Input shape: [1 6 2]
Output shape: [1 2]


In [140]:
print("File size without ptq: ", os.path.getsize("Lab4/models/model_{}.tflite".format("cnn_3")))

File size without ptq:  70492


In [14]:
test_ds = tf.data.experimental.load('./Lab4/th_test', tensor_specs)
test_data = test_ds.map(lambda a, b: a)
test_labels = test_ds.map(lambda a, b: b)

In [79]:
test_data = test_data.unbatch().batch(1)

In [ ]:
test_labels = np.array(list(test_labels.unbatch().as_numpy_iterator())) # this fits in memory

Let's generate np arrays

In [100]:
def predict_th(interpreter, test_data):
    
    for i, window in enumerate(test_data.as_numpy_iterator()):
        interpreter.set_tensor(input_details[0]['index'], window)
        interpreter.invoke()
        predicted = interpreter.get_tensor(output_details[0]['index'])

        if i==0:
            preds = np.array(predicted, dtype=np.float32)
        else:
            preds = np.append(preds, predicted, axis=0)
        
    return preds

TRY: np.array(list(dataset.as_numpy_iterator())) 

In [96]:
def calculateMAE(actuals, preds):
    """
    Calculate MAE between 2 np vectors.
    np: numpy

    """
    return np.mean(np.absolute(actuals - preds))

In [141]:
preds = predict_th(interpreter, test_data)

In [142]:
print("Temperature MAE: ", calculateMAE(test_labels[:, 0], preds[:, 0]))
print("Humidity MAE: ", calculateMAE(test_labels[:, 1], preds[:, 1]))

Temperature MAE:  0.25623897
Humidity MAE:  0.7927708


**Now let's move on MFCCs**

In [34]:
from Lab3.lab3_ex2_working_version import SignalGenerator

In [52]:
#from Lab3.lab3_ex2_working_version import SignalGenerator

data_dir = '/home/pi/WORK_DIR/notebooks/Ex_Gianlu/LABs/LAB 3/data/mini_speech_commands'

filenames = tf.io.gfile.glob(data_dir + '/*/*')  # Return list of files matching the pattern
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

test_files = filenames[int(num_samples * 0.9):]

LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']
LABELS = np.delete(LABELS, 0)

In [54]:
#STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
                'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
                'num_coefficients': 10}

options = MFCC_OPTIONS

generator = SignalGenerator(LABELS, 16000, **options)
test_ds = generator.make_dataset(test_files, False)

Audio shape without padding (None,)
Audio shape with padding (16000,)


In [55]:
test_data = test_ds.map(lambda a, b: a)
test_labels = test_ds.map(lambda a, b: b)

In [64]:
test_labels = np.array(list(test_labels.unbatch().as_numpy_iterator())) # this fits in memory
test_data = test_data.unbatch().batch(1)

Let's test inference with tflite model:

In [38]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/model_{}.tflite".format("cnn_mfccs"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_conv2d_input:0
Input shape: [ 1 49 10  1]
Output shape: [1 8]


In [89]:
def predict_from_mfccs(interpreter, test_data):
    for i, mfccs in enumerate(test_data.as_numpy_iterator()):
        interpreter.set_tensor(input_details[0]['index'], mfccs)
        interpreter.invoke()
        predicted = interpreter.get_tensor(output_details[0]['index'])

        if i==0:
            preds = np.array(predicted, dtype=np.float32)
        else:
            preds = np.append(preds, predicted, axis=0)

    preds = np.argmax(preds, axis=1)
    
    return preds

In [75]:
def compute_accuracy(true_values, predictions):
    N = true_values.shape[0]
    return (true_values == predictions).sum() / N

In [77]:
compute_accuracy(test_labels, preds)

0.1775

# 1.3

In [ ]:
# Run only to convert:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()


In [123]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/model_{}.tflite".format("CNN_opt_ptq"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: conv1d_input
Input shape: [1 6 2]
Output shape: [1 2]


In [85]:
print("File size after ptq: ", os.path.getsize("Lab4/models/model_{}.tflite".format("CNN_opt_ptq")))

File size after ptq:  23952


In [127]:
tensor_specs = (tf.TensorSpec([None, 6, 2], dtype=tf.float32),
                tf.TensorSpec([None, 2]))
test_ds = tf.data.experimental.load('Lab4/th_test', tensor_specs)

In [128]:
test_data = test_ds.map(lambda a, b: a)
test_labels = test_ds.map(lambda a, b: b)
test_data = test_data.unbatch().batch(1)

Latency and MAE for CNN:

In [108]:
a = time.time()
preds = predict_th(interpreter, test_data)
b = time.time()
test_labels = np.array(list(test_labels.unbatch().as_numpy_iterator()))

In [112]:
temp_mae_pqt = calculateMAE(test_labels[:, 0], preds[:, 0])
hum_mae_pqt = calculateMAE(test_labels[:, 1], preds[:, 1])
print("Temperature MAE: ", temp_mae_pqt)
print("Humidity MAE: ", hum_mae_pqt)
print("Latency for inference: {} sec".format(b-a))

Temperature MAE:  0.93616277
Humidity MAE:  0.95842415
Latency for inference: 31.048535346984863 sec


# 1.5

**TEMP AND HUM**

In [ ]:
# Only to convert in tf lite weight + activ. optimization:
def representative_dataset_gen():
     for x, _ in train_ds.unbatch().batch(1).take(1000):
         yield [x]
        
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

In [131]:
print("File size after ptq weights and activations: ", os.path.getsize("Lab4/models/{}.tflite".format("cnn_weight_act_opt_th")))

File size after ptq weights and activations:  21752


In [132]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/{}.tflite".format("cnn_weight_act_opt_th"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_x:0
Input shape: [1 6 2]
Output shape: [1 2]


In [133]:
a = time.time()
preds = predict_th(interpreter, test_data)
b = time.time()
#test_labels = np.array(list(test_labels.unbatch().as_numpy_iterator()))

In [134]:
temp_mae_weights_act_cnn = calculateMAE(test_labels[:, 0], preds[:, 0])
hum_mae_weights_act_cnn = calculateMAE(test_labels[:, 1], preds[:, 1])
print("Temperature MAE: ", temp_mae_pqt)
print("Humidity MAE: ", hum_mae_pqt)
print("Latency for inference: {} sec".format(b-a))

Temperature MAE:  0.93616277
Humidity MAE:  0.95842415
Latency for inference: 30.628411531448364 sec


MLP:

In [143]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/{}.tflite".format("mlp_weight_act_opt_th"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_x:0
Input shape: [1 6 2]
Output shape: [1 2]


In [144]:
print("File size after ptq weights and activations: ", os.path.getsize("Lab4/models/{}.tflite".format("mlp_weight_act_opt_th")))

File size after ptq weights and activations:  21680


In [145]:
a = time.time()
preds = predict_th(interpreter, test_data)
b = time.time()
#test_labels = np.array(list(test_labels.unbatch().as_numpy_iterator()))
temp_mae_weights_act_cnn = calculateMAE(test_labels[:, 0], preds[:, 0])
hum_mae_weights_act_cnn = calculateMAE(test_labels[:, 1], preds[:, 1])
print("Temperature MAE: ", temp_mae_pqt)
print("Humidity MAE: ", hum_mae_pqt)
print("Latency for inference: {} sec".format(b-a))

Temperature MAE:  0.93616277
Humidity MAE:  0.95842415
Latency for inference: 30.431209325790405 sec


LSTM: 

In [147]:
interpreter = tf.lite.Interpreter(model_path="Lab4/models/{}.tflite".format("lstm_weight_act_opt_th"))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

print("File size after ptq weights and activations: ", os.path.getsize("Lab4/models/{}.tflite".format("lstm_weight_act_opt_th")))

a = time.time()
preds = predict_th(interpreter, test_data)
b = time.time()
#test_labels = np.array(list(test_labels.unbatch().as_numpy_iterator()))
temp_mae_weights_act_cnn = calculateMAE(test_labels[:, 0], preds[:, 0])
hum_mae_weights_act_cnn = calculateMAE(test_labels[:, 1], preds[:, 1])
print("Temperature MAE: ", temp_mae_pqt)
print("Humidity MAE: ", hum_mae_pqt)
print("Latency for inference: {} sec".format(b-a))

RuntimeError: tensorflow/lite/kernels/unidirectional_sequence_lstm.cc:293 input->type != kTfLiteFloat32 (INT8 != FLOAT32)Node number 1 (UNIDIRECTIONAL_SEQUENCE_LSTM) failed to prepare.
tensorflow/lite/kernels/unidirectional_sequence_lstm.cc:293 input->type != kTfLiteFloat32 (INT8 != FLOAT32)Node number 1 (UNIDIRECTIONAL_SEQUENCE_LSTM) failed to prepare.
